# Getting Gun Data

The idea here is to grab data related to guns, namely:

* How many people are victims of gun violence in each year, in each state?
* What is the NRA rating for each senator in each (election) year?
* Which senators get NRA endorsements each year?

In [34]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re

## Part 1: Get Gun Violence Data

This is from a .csv file I got from the CDC....https://wonder.cdc.gov/controller/datarequest/D76;jsessionid=9F60FA0EE42118E226739764CB333F97

I want to compare numbers with the data from this [site](https://everytownresearch.org/gun-violence-by-the-numbers/)

For instance, there should be 13,000 gun homicides per year

In [35]:
# Load the data
gun_deaths = pd.read_table("./Underlying_Cause_of_Death_1999-2016.txt")
print(gun_deaths.info())
print(gun_deaths.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 8 columns):
Notes         69 non-null object
Year          827 non-null float64
Year Code     827 non-null float64
State         827 non-null object
State Code    827 non-null float64
Deaths        827 non-null float64
Population    827 non-null float64
Crude Rate    827 non-null object
dtypes: float64(5), object(3)
memory usage: 56.1+ KB
None
  Notes    Year  Year Code       State  State Code  Deaths  Population  \
0   NaN  1999.0     1999.0     Alabama         1.0   293.0   4430141.0   
1   NaN  1999.0     1999.0      Alaska         2.0    27.0    624779.0   
2   NaN  1999.0     1999.0     Arizona         4.0   303.0   5023823.0   
3   NaN  1999.0     1999.0    Arkansas         5.0   118.0   2651860.0   
4   NaN  1999.0     1999.0  California         6.0  1380.0  33499204.0   

  Crude Rate  
0        6.6  
1        4.3  
2        6.0  
3        4.4  
4        4.1  


Let's check with those per-year stats to see if it holds:

In [19]:
# Group by year
print(gun_deaths.groupby('Year')['Deaths'].sum())
print("Mean since 1999: {:0.0f}".format(gun_deaths.groupby('Year')['Deaths'].sum().mean()))

Year
1999.0    10792.0
2000.0    10760.0
2001.0    11296.0
2002.0    11788.0
2003.0    11888.0
2004.0    11596.0
2005.0    12314.0
2006.0    12765.0
2007.0    12592.0
2008.0    12161.0
2009.0    11482.0
2010.0    11045.0
2011.0    11044.0
2012.0    11593.0
2013.0    11171.0
2014.0    10965.0
2015.0    12973.0
2016.0    14394.0
Name: Deaths, dtype: float64
Mean since 1999: 11812


These are a bit low, but they're pretty in line with what I'm looking for here. I think there might be some "suppressed" data; I'm not 100% sure. But this is data for Assault via discharge of firearms....on to the NRA data!

## Part 2: Find NRA Ratings for each Senator by (election) Year

There's a couple possible sources here:

* [NRA site](https://www.nrapvf.org/grades/) (only problem: need membership to access archives!)
* [VoteSmart](https://votesmart.org/interest-group/1034/national-rifle-association) (problem: need to pay at least $45 to access API)

Based on this, I'm going to have to manually scrape VoteSmart; however, their URLs are somewhat nonsensical...I'll just hardcode them:



In [58]:
# Put together all the URLs
# https://votesmart.org/interest-group/1034/rating/10252?p=2&of=#
nra_2018 = 'https://votesmart.org/interest-group/1034/rating/10252'
nra_2017 = 'https://votesmart.org/interest-group/1034/rating/9703'
nra_2016 = 'https://votesmart.org/interest-group/1034/rating/10178'
nra_2015 = 'https://votesmart.org/interest-group/1034/rating/7776'
nra_2014 = 'https://votesmart.org/interest-group/1034/rating/7165'
nra_2013 = 'https://votesmart.org/interest-group/1034/rating/8738'
nra_2012 = 'https://votesmart.org/interest-group/1034/rating/8741'
nra_2010 = 'https://votesmart.org/interest-group/1034/rating/8740'
nra_2008 = 'https://votesmart.org/interest-group/1034/rating/8787'
nra_2006 = 'https://votesmart.org/interest-group/1034/rating/8786'
nra_2004 = 'https://votesmart.org/interest-group/1034/rating/8785'
nra_2002 = 'https://votesmart.org/interest-group/1034/rating/8784'
nra_2000 = 'https://votesmart.org/interest-group/1034/rating/8788'
nra_1998 = 'https://votesmart.org/interest-group/1034/rating/8739'
nra_1994 = 'https://votesmart.org/interest-group/1034/rating/82'

all_nra = [nra_2018, nra_2017, nra_2016,
           nra_2015, nra_2014, nra_2013,
           nra_2012, nra_2010, nra_2008,
           nra_2006, nra_2004, nra_2002,
           nra_2000, nra_1998, nra_1994]


Now test out with 2018 dataset

In [54]:
# See the results 
results = requests.get(nra_2018).text

In [55]:
# Try grabbing the list of items
nra_soup = BeautifulSoup(results, 'html.parser')
nra_pages = nra_soup.find('ul', {"class" : 'range'}).find_all('li')
all_pages = [item.text.strip() for item in nra_pages]
print(all_pages)

['1', '2', '3']


In [56]:
# Grab the headline including the year
year_headline = nra_soup.find('h4').text
print(re.findall(r'\d{4}',year_headline)[0])

2018


I've now demonstrated that I can grab the pages; and "1" actually works for the main page too! Now to make sure I can pull the table correctly:

In [57]:
nra_table = nra_soup.find('table').find_all('tr')
for row in nra_table[1:5]:
    print(row.find_all('td')[0].text) # State
    print(row.find_all('td')[1].text) # Office
    print(row.find_all('td')[2].text) # District
    print(row.find_all('td')[3].text) # Name
    print(row.find_all('td')[4].text) # Party
    print(row.find_all('td')[5].text) # Rating   

GA
State House
111
Geoff Cauble
Republican
86%
GA
State Senate
17
Brian Strickland
Republican
93%
IL
U.S. House
12
Michael Bost
Republican
93%
IL
U.S. House
16
Adam Kinzinger
Republican
93%


I've now demonstrated that I can do any of these; I'm going to wrap them into functions

In [106]:
# Function for pulling out data for a given page/table
def pullNraInfo(page):
    
    # Turn the page into soup
    results = requests.get(page).text
    nra_soup = BeautifulSoup(results, 'html.parser')
    
    # Find the table
    nra_table = nra_soup.find('table').find_all('tr')[1:]
    
    # Pull out the useful quantities
    state = [row.find_all('td')[0].text for row in nra_table]
    office = [row.find_all('td')[1].text for row in nra_table]
    name = [row.find_all('td')[3].text for row in nra_table]
    rating = [row.find_all('td')[5].text.strip('%') for row in nra_table]
    
    # Combine into a data frame
    nra_df = pd.DataFrame({'State' : state,
                           'Office' : office,
                           'Name' : name,
                           'Rating' : rating})
    
    # Pull the year from the headline
    year_headline = nra_soup.find('h4').text
    this_year = re.findall(r'\d{4}',year_headline)[0]
    
    # Broadcast it
    nra_df['Year'] = int(this_year)
    
    return nra_df

In [107]:
# Test it out
test = pullNraInfo(nra_2018)
print(test.head())
print(test.info())

               Name        Office Rating State  Year
0      Geoff Cauble   State House     86    GA  2018
1  Brian Strickland  State Senate     93    GA  2018
2      Michael Bost    U.S. House     93    IL  2018
3    Adam Kinzinger    U.S. House     93    IL  2018
4      Darin LaHood    U.S. House     93    IL  2018
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
Name      200 non-null object
Office    200 non-null object
Rating    200 non-null object
State     200 non-null object
Year      200 non-null int64
dtypes: int64(1), object(4)
memory usage: 7.9+ KB
None


This works great for 1 page, but I need to assemble all the pages. I need to make multiple URLs per year, specifically:

* Take the given URL for the year
* Detect how many pages there are (or at least list them)
* Create an augmented URL for each one
* Output the list of URLs for each year

This should be relatively simple; I'll try for 2018

In [60]:
# Recall how to get 2018 pages
results = requests.get(nra_2018).text
nra_soup = BeautifulSoup(results, 'html.parser')
nra_pages = nra_soup.find('ul', {"class" : 'range'}).find_all('li')
all_pages = [item.text.strip() for item in nra_pages]
print(all_pages)

# Add the page and office number for each:
all_2018_urls = [nra_2018 + '?p={}&of=#'.format(item) for item in all_pages]
print(all_2018_urls)

['1', '2', '3']
['https://votesmart.org/interest-group/1034/rating/10252?p=1&of=#', 'https://votesmart.org/interest-group/1034/rating/10252?p=2&of=#', 'https://votesmart.org/interest-group/1034/rating/10252?p=3&of=#']


Now to really test, grab the data from all 3 pages

In [65]:
# Use the fxn above
all_2018_dfs = [pullNraInfo(url) for url in all_2018_urls]

# Concatenate and find info
full_2018 = pd.concat(all_2018_dfs)
print(full_2018.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 16
Data columns (total 5 columns):
Name      417 non-null object
Office    417 non-null object
Rating    417 non-null object
State     417 non-null object
Year      417 non-null int64
dtypes: int64(1), object(4)
memory usage: 19.5+ KB
None


This is all great! so now I just need to make that a function and test it again, then I can apply it to ALL the URLs    

In [95]:
# Function for getting all urls for a given year
def expandUrls(orig_url):
    
    # Get the original url
    results = requests.get(orig_url).text
    nra_soup = BeautifulSoup(results, 'html.parser')
    
    # Use try/except to find extra pages
    try:
        nra_pages = nra_soup.find('ul', {"class" : 'range'}).find_all('li')
        all_pages = [item.text.strip() for item in nra_pages]
    
        # Find the max page and make a range up to there
        max_page = int(all_pages[-1])
        print(max_page)
        page_nums = list(range(1, max_page + 1))
      
        # Add the page and office number for each:
        all_urls = [orig_url + '?p={}&of=#'.format(item) for item in page_nums]
    
    # Grab exception: 1 page
    except:
        all_urls = [orig_url]
        print(1)
        
    return all_urls

In [96]:
# Test it
expandUrls(nra_2018)

3


['https://votesmart.org/interest-group/1034/rating/10252?p=1&of=#',
 'https://votesmart.org/interest-group/1034/rating/10252?p=2&of=#',
 'https://votesmart.org/interest-group/1034/rating/10252?p=3&of=#']

In [97]:
type(expandUrls(nra_2013))

1


list

Now for the full workflow:

* Expand all the URLs to the full set (and check how many there are!)
* For each URL, grab the rating info
* Concatenate the data frames to get 1 big one

In [98]:
# Quick for-loop to get all the urls
all_urls = []
# Go until 2004
for year in all_nra: 
    # Get the urls for that year
    all_urls += expandUrls(year)
    
print(len(all_urls))

3
6
68
6
3
1
39
4
3
3
3
3
3
2
2
149


I have all the URLs now; I now need to scrape all 150ish URLs for their data

In [108]:
%%time
# Try a list comprehension to get 201 data frames
full_nra_df = [pullNraInfo(url) for url in all_urls]

CPU times: user 18.6 s, sys: 225 ms, total: 18.8 s
Wall time: 12min 26s


That took a little while (I think like 5-10 min); I'm going to collapse this into one data frame, then pickle it so I don't have to do this again

In [109]:
# Concatenate everything
nra_all_years = pd.concat(full_nra_df)

# Pickle it
nra_all_years.to_pickle('nra_all_years.pickle')

In [114]:
nra_all_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28089 entries, 0 to 192
Data columns (total 5 columns):
Name      28089 non-null object
Office    28089 non-null object
Rating    28089 non-null object
State     28089 non-null object
Year      28089 non-null int64
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [115]:
# Screen out only senators for US
nra_all_years[nra_all_years.Office == 'U.S. Senate'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 15 to 65
Data columns (total 5 columns):
Name      244 non-null object
Office    244 non-null object
Rating    244 non-null object
State     244 non-null object
Year      244 non-null int64
dtypes: int64(1), object(4)
memory usage: 11.4+ KB
